Vizualizácia dát v Bratislave - staré mesto:    

In [26]:
import pandas as pd
import random

# náhodné dáta na vizualizáciu znečistenia ovzdušia na základe senzorom uložených pri vstupe do mesta
data = {
    "Latitude": [48.1423, 48.1432, 48.1418, 48.1439] * 14,
    "Longitude": [17.1082, 17.1093, 17.1098, 17.1102] * 14,
    "Hour": [h for h in range(6, 20) for _ in range(4)],
     "Pollution_Level": [random.randint(1, 512) for _ in range(56)]
}

latitude_length = len(data["Latitude"])
longitude_length = len(data["Longitude"])
hour_length = len(data["Hour"])
pollution_level_length = len(data["Pollution_Level"])

df = pd.DataFrame(data)
print(data["Hour"])

[6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 10, 11, 11, 11, 11, 12, 12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 14, 15, 15, 15, 15, 16, 16, 16, 16, 17, 17, 17, 17, 18, 18, 18, 18, 19, 19, 19, 19]


In [39]:
# Create a base map centered around Bratislava's Old Town
import folium
from folium.plugins import HeatMap

m = folium.Map(location=[48.1425, 17.1092], zoom_start=20)

heatmap_data = [
    [row["Latitude"], row["Longitude"], row["Pollution_Level"]] for idx, row in df.iterrows()
]

# Add the heatmap to the map
HeatMap(heatmap_data).add_to(m)

m.save("bratislava_pollution_heatmap.html")

Animacia:

In [20]:
import folium
from folium.plugins import HeatMapWithTime

#Mapa - zoom na stare mesto
m = folium.Map(location=[48.1425, 17.1092], zoom_start=16)

# Organize data for the heatmap animation
hourly_data = []
for hour in range(6, 20): # od 06:00 do 19:00
    hourly_df = df[df["Hour"] == hour]
    hourly_list = [[row["Latitude"], row["Longitude"], row["Pollution_Level"]] for idx, row in hourly_df.iterrows()]
    hourly_data.append(hourly_list)

HeatMapWithTime(hourly_data, auto_play=True, max_opacity=0.8, use_local_extrema=True,radius=99).add_to(m)

m.save("bratislava_pollution_hodinova.html")

Zmena na frames do prezentacie:

In [40]:
import numpy as np
import pandas as pd
import folium
from folium.plugins import HeatMap
import branca.colormap as cm
import matplotlib.pyplot as plt
import os
from matplotlib import colors as mcolors

m = folium.Map(location=[48.1425, 17.1090], zoom_start=20, control_scale=True)

# Define the color map
colors = list(plt.cm.inferno(np.linspace(0, 1, 256)))
colors = [mcolors.rgb2hex(c) for c in colors]
colormap = cm.StepColormap(colors, vmin=min(df['Pollution_Level']), vmax=max(df['Pollution_Level']), index=np.linspace(min(df['Pollution_Level']), max(df['Pollution_Level']), 256))

m.add_child(colormap)

frames_folder = "frames"
if not os.path.exists(frames_folder):
    os.makedirs(frames_folder)

# Generate heatmap frames for each hour
for hour in df['Hour'].unique():
    data_hour = df[df['Hour'] == hour][['Latitude', 'Longitude', 'Pollution_Level']].groupby(['Latitude', 'Longitude']).mean().reset_index().values.tolist()
    
    HeatMap(data_hour, radius=30, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'yellow', 1: 'red'}, max_val=float(max(df['Pollution_Level'])), min_val=float(min(df['Pollution_Level']))).add_to(m)
    
    # Check if the title exists, replace it; otherwise, add it
    if 'title' in m.get_root().html._children:
        m.get_root().html._children['title'] = folium.Element(f'<h3 align="center" style="font-size:20px"><b>Znečistenie ovzdušia o: {hour}:00</b></h3>')
    else:
        title_html = f'<h3 align="center" style="font-size:20px"><b>Znečistenie ovzdušia o: {hour}:00</b></h3>'
        m.get_root().html.add_child(folium.Element(title_html), name='title')
    
    m.save(os.path.join(frames_folder, f"hour_{hour}.html"))
    for child in list(m._children):
        if child.startswith('HeatMap'):
            del m._children[child]

print(f"Frames saved in the '{frames_folder}' directory.")


C:\Users\henox\AppData\Local\Temp\ipykernel_32780\792256551.py:27: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  HeatMap(data_hour, radius=30, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'yellow', 1: 'red'}, max_val=float(max(df['Pollution_Level'])), min_val=float(min(df['Pollution_Level']))).add_to(m)
C:\Users\henox\AppData\Local\Temp\ipykernel_32780\792256551.py:27: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  HeatMap(data_hour, radius=30, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'yellow', 1: 'red'}, max_val=float(max(df['Pollution_Level'])), min_val=float(min(df['Pollution_Level']))).add_to(m)
C:\Users\henox\AppData\Local\Temp\ipykernel_32780\792256551.py:27: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  HeatMap(data_hour, radius=30, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'yellow', 

Frames saved in the 'frames' directory.


C:\Users\henox\AppData\Local\Temp\ipykernel_32780\792256551.py:27: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  HeatMap(data_hour, radius=30, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'yellow', 1: 'red'}, max_val=float(max(df['Pollution_Level'])), min_val=float(min(df['Pollution_Level']))).add_to(m)
